# imports

In [1]:
import pandas as pd
import os

# read data

## view crimes and save crimes and files in a dict

In [36]:
crimes = os.listdir('police_data')
crimes.remove('tree_file.txt')
crime_files = {i:os.listdir(os.path.join('police_data',i)) for i in crimes}
crimes

['abigeato',
 'amenazas',
 'delitos_sexuales',
 'extorsion',
 'homicidios',
 'homicidios_accidente_de_transito',
 'hurto_a_comercio',
 'hurto_a_entidades_financieras',
 'hurto_a_personas',
 'hurto_a_residencias',
 'hurto_motocicletas',
 'hurto_vehiculos',
 'lesiones_en_accidente_de_transito',
 'lesiones_personales',
 'pirateria_terrestre',
 'secuestro',
 'terrorismo',
 'violencia_intrafamiliar']

## function for trans

In [126]:
#for the dane code
def clean_cod_dane(cod):
    if len(cod) == 7:
        return '0'+ cod
    else:
        return cod

    
#only read for column names    
def read_dataset(path):
    df = pd.read_excel(path,nrows=30)
    
    #get first element
    first_reg = [str(n).upper() for n in list(df.iloc[:20,0])].index('DEPARTAMENTO')
    
    df = pd.read_excel(path, 
                   skiprows=first_reg+1, #ignore header
                    nrows=30,
                   dtype={'CODIGO DANE':str}#,'CANTIDAD':int}
                  )
    
    # rename cols
    df.columns = [n.upper().replace('*','').replace(' ','_') for n in df.columns]
    
    return df


#read and transform
def read_n_transform_dataset(path):
    df = pd.read_excel(path,nrows=30)
    
    #get first element
    first_reg = [str(n).upper() for n in list(df.iloc[:20,0])].index('DEPARTAMENTO')
    
    df = pd.read_excel(path, 
                   skiprows=first_reg+1, #ignore header
                   dtype={'CODIGO DANE':str,'DEPARTAMENTO':str}#,'CANTIDAD':int}
                  )
    
    # rename cols
    df.columns = [n.upper()\
                  .replace('*','')\
                  .replace(' ','_')\
                  .replace('AGRUPA_EDAD_PERSONA','GRUPO_ETARIO') for n in df.columns]
    
    #drop last info
    last_reg = [str(i).strip() for i in list(df.DEPARTAMENTO)].index('TOTAL')
    df = df.iloc[:last_reg,:]
    
    #correct DANE code
    df['CODIGO_DANE'] = df['CODIGO_DANE'].apply(lambda x: clean_cod_dane(str(x)))
    df['CANTIDAD'] = df['CANTIDAD'].apply(lambda x: int(x))
    return df

## review the columns

In [78]:
for crime,files in crime_files.items():
#     _df = read_dataset(os.path.join('police_data',k,v[0]))
    for file in files:
        _df = read_dataset(os.path.join('police_data',crime,file))
        print(crime+' '+file+' : ')
        print(_df.columns)
    print('\n')

abigeato 2010.xlsx : 
Index(['DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'ARMAS_MEDIOS',
       'FECHA_HECHO', 'GENERO', 'AGRUPA_EDAD_PERSONA', 'CANTIDAD'],
      dtype='object')
abigeato 2011.xlsx : 
Index(['DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'ARMAS_MEDIOS',
       'FECHA_HECHO', 'GENERO', 'AGRUPA_EDAD_PERSONA', 'CANTIDAD'],
      dtype='object')
abigeato 2012.xlsx : 
Index(['DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'ARMAS_MEDIOS',
       'FECHA_HECHO', 'GENERO', 'AGRUPA_EDAD_PERSONA', 'CANTIDAD'],
      dtype='object')
abigeato 2013.xlsx : 
Index(['DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'ARMAS_MEDIOS',
       'FECHA_HECHO', 'GENERO', 'AGRUPA_EDAD_PERSONA', 'CANTIDAD'],
      dtype='object')
abigeato 2014.xlsx : 
Index(['DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'ARMAS_MEDIOS',
       'FECHA_HECHO', 'GENERO', 'AGRUPA_EDAD_PERSONA', 'CANTIDAD'],
      dtype='object')
abigeato 2015.xlsx : 
Index(['DEPARTAMENTO', 'MUNICIPIO', 'CODIGO_DANE', 'ARMAS_MEDIOS',
       'FECHA_HE

## read and transform

In [133]:
crime_dfs = {}
for crime,files in crime_files.items():
    df_list = []
    if crime in ['abigeato','amenazas']:
        continue
    for file in files:
        print(f'reading: {crime}/{file}')
        _df = read_n_transform_dataset(os.path.join('police_data',crime,file))
        print(f'{crime}/{file} succesfully read')
        df_list.append(_df)
    crime_dfs[crime] = pd.concat(df_list)
    crime_dfs[crime].to_csv(f'police_data/compilate_by_crime/{crime}.csv',index=False,encoding='utf8',sep=';')
    print(f'{crime} succesfully saved')
    print('\n')

reading: abigeato/2010.xlsx
abigeato/2010.xlsx succesfully read
reading: abigeato/2011.xlsx
abigeato/2011.xlsx succesfully read
reading: abigeato/2012.xlsx
abigeato/2012.xlsx succesfully read
reading: abigeato/2013.xlsx
abigeato/2013.xlsx succesfully read
reading: abigeato/2014.xlsx
abigeato/2014.xlsx succesfully read
reading: abigeato/2015.xlsx
abigeato/2015.xlsx succesfully read
reading: abigeato/2016.xlsx
abigeato/2016.xlsx succesfully read
reading: abigeato/2017.xlsx
abigeato/2017.xlsx succesfully read
reading: abigeato/2018.xlsx
abigeato/2018.xlsx succesfully read
reading: abigeato/2019.xlsx
abigeato/2019.xlsx succesfully read
reading: abigeato/2020.xls
abigeato/2020.xls succesfully read
abigeato succesfully saved


reading: amenazas/2010.xlsx
amenazas/2010.xlsx succesfully read
reading: amenazas/2011.xlsx
amenazas/2011.xlsx succesfully read
reading: amenazas/2012.xlsx
amenazas/2012.xlsx succesfully read
reading: amenazas/2013.xlsx
amenazas/2013.xlsx succesfully read
reading: amen

## review an error

In [176]:
for k,v in crime_dfs.items():
    
    if 'ESCOPOLAMINA' in list(v.CODIGO_DANE):
        print(k)
    else:
        continue
crime_dfs['hurto_a_comercio'][crime_dfs['hurto_a_comercio']['CODIGO_DANE'] == 'ESCOPOLAMINA'] 
the output was hurto_comercios in 2013

hurto_a_personas
violencia_intrafamiliar


In [177]:
crime_dfs['hurto_a_comercio'][crime_dfs['hurto_a_comercio']['CODIGO_DANE'] == 'ESCOPOLAMINA']

,DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,ARMAS_MEDIOS,FECHA_HECHO,GENERO,GRUPO_ETARIO,CANTIDAD
250427,NARIÑO,NO REGISTRA,NO REGISTRA,SIN EMPLEO DE ARMAS,2019-11-04,FEMENINO,ADULTOS,1


In [178]:
# se equivocaron en el hurto a comercios 2013
for crime,files in crime_files.items():
    df_list = []
    if crime != 'hurto_a_comercio':
        continue
    for file in files:
        print(f'reading: {crime}/{file}')
        _df = read_n_transform_dataset(os.path.join('police_data',crime,file))
        print(f'{crime}/{file} succesfully read')
        df_list.append(_df)
    crime_dfs[crime] = pd.concat(df_list)
    crime_dfs[crime].to_csv(f'police_data/compilate_by_crime/{crime}.csv',index=False,encoding='utf8',sep=';')
    print(f'{crime} succesfully saved')
    print('\n')

reading: hurto_a_comercio/2010.xlsx
hurto_a_comercio/2010.xlsx succesfully read
reading: hurto_a_comercio/2011.xlsx
hurto_a_comercio/2011.xlsx succesfully read
reading: hurto_a_comercio/2012.xlsx
hurto_a_comercio/2012.xlsx succesfully read
reading: hurto_a_comercio/2013.xlsx
hurto_a_comercio/2013.xlsx succesfully read
reading: hurto_a_comercio/2014.xlsx
hurto_a_comercio/2014.xlsx succesfully read
reading: hurto_a_comercio/2015.xlsx
hurto_a_comercio/2015.xlsx succesfully read
reading: hurto_a_comercio/2016.xlsx
hurto_a_comercio/2016.xlsx succesfully read
reading: hurto_a_comercio/2017.xlsx
hurto_a_comercio/2017.xlsx succesfully read
reading: hurto_a_comercio/2018.xlsx
hurto_a_comercio/2018.xlsx succesfully read
reading: hurto_a_comercio/2019.xlsx
hurto_a_comercio/2019.xlsx succesfully read
reading: hurto_a_comercio/2020.xls
hurto_a_comercio/2020.xls succesfully read
hurto_a_comercio succesfully saved




## view 1

In [137]:
crime_dfs['abigeato']

,DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,ARMAS_MEDIOS,FECHA_HECHO,GENERO,GRUPO_ETARIO,CANTIDAD
0,HUILA,CAMPOALEGRE,41132000,NO REPORTADO,2010-01-01,FEMENINO,ADULTOS,1
1,META,SAN JUAN DE ARAMA,50683000,NO REPORTADO,2010-01-01,MASCULINO,ADULTOS,1
2,BOYACÁ,BELÉN,15087000,NO REPORTADO,2010-01-02,FEMENINO,ADULTOS,1
3,CASANARE,PORE,85263000,NO REPORTADO,2010-01-02,MASCULINO,ADULTOS,1
4,CALDAS,NEIRA,17486000,NO REPORTADO,2010-01-03,MASCULINO,ADULTOS,1
...,...,...,...,...,...,...,...,...
2234,HUILA,PITALITO,41551000,SIN EMPLEO DE ARMAS,2020-12-27,MASCULINO,ADULTOS,1
2235,ATLÁNTICO,SABANALARGA,08638000,SIN EMPLEO DE ARMAS,2020-12-28,MASCULINO,ADULTOS,1
2236,SUCRE,EL ROBLE,70233000,SIN EMPLEO DE ARMAS,2020-12-28,MASCULINO,ADULTOS,1
2237,CESAR,AGUACHICA,20011000,ARMA DE FUEGO,2020-12-31,MASCULINO,ADULTOS,1


In [141]:
crime_dfs['abigeato'].groupby(['DEPARTAMENTO','MUNICIPIO'])['CANTIDAD'].sum().reset_index().sort_values('CANTIDAD', ascending=False)

,DEPARTAMENTO,MUNICIPIO,CANTIDAD
682,META,VILLAVICENCIO (CT),583
416,CESAR,VALLEDUPAR (CT),490
358,CASANARE,YOPAL (CT),384
432,CUNDINAMARCA,BOGOTÁ D.C. (CT),352
946,VALLE,CALI (CT),317
...,...,...,...
771,QUINDÍO,BUENAVISTA,1
194,BOYACÁ,BUSBANZÁ,1
309,CALDAS,MARMATO,1
302,CALDAS,BELALCÁZAR,1


# Add the crime and Merge all

In [183]:
for crime,c_df in crime_dfs.items():
    c_df['CRIME'] = crime  

In [184]:
full_df = pd.concat(crime_dfs.values())

In [185]:
full_df.head()

,DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,ARMAS_MEDIOS,FECHA_HECHO,GENERO,GRUPO_ETARIO,CANTIDAD,CRIME,DESCRIPCION_CONDUCTA
0,HUILA,CAMPOALEGRE,41132000,NO REPORTADO,2010-01-01 00:00:00,FEMENINO,ADULTOS,1,abigeato,NaN
1,META,SAN JUAN DE ARAMA,50683000,NO REPORTADO,2010-01-01 00:00:00,MASCULINO,ADULTOS,1,abigeato,NaN
2,BOYACÁ,BELÉN,15087000,NO REPORTADO,2010-01-02 00:00:00,FEMENINO,ADULTOS,1,abigeato,NaN
3,CASANARE,PORE,85263000,NO REPORTADO,2010-01-02 00:00:00,MASCULINO,ADULTOS,1,abigeato,NaN
4,CALDAS,NEIRA,17486000,NO REPORTADO,2010-01-03 00:00:00,MASCULINO,ADULTOS,1,abigeato,NaN


In [188]:
full_df.dtypes

DEPARTAMENTO            object
MUNICIPIO               object
CODIGO_DANE             object
ARMAS_MEDIOS            object
FECHA_HECHO             object
GENERO                  object
GRUPO_ETARIO            object
CANTIDAD                 int64
CRIME                   object
DESCRIPCION_CONDUCTA    object
dtype: object

In [199]:
full_df.to_csv('police_data/all_data.csv',index=False,encoding='utf8',sep=';')

In [193]:
full_df.DESCRIPCION_CONDUCTA.unique()

array([nan, 'ARTÍCULO 169. SECUESTRO EXTORSIVO',
       'ARTÍCULO 168. SECUESTRO SIMPLE'], dtype=object)

In [200]:
'2010-07-23 00:00:00'[:10]

'2010-07-23'

In [204]:
full_df.dtypes

DEPARTAMENTO            object
MUNICIPIO               object
CODIGO_DANE             object
ARMAS_MEDIOS            object
FECHA_HECHO             object
GENERO                  object
GRUPO_ETARIO            object
CANTIDAD                 int64
CRIME                   object
DESCRIPCION_CONDUCTA    object
FECHA                   object
dtype: object

In [203]:
full_df['FECHA'] = full_df['FECHA_HECHO'].apply(lambda x: pd.to_datetime(str(x)[:10], format='%Y-%m-%d', errors='ignore'))

In [205]:
full_df.head()

,DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,ARMAS_MEDIOS,FECHA_HECHO,GENERO,GRUPO_ETARIO,CANTIDAD,CRIME,DESCRIPCION_CONDUCTA,FECHA
0,HUILA,CAMPOALEGRE,41132000,NO REPORTADO,2010-01-01 00:00:00,FEMENINO,ADULTOS,1,abigeato,NaN,2010-01-01 00:00:00
1,META,SAN JUAN DE ARAMA,50683000,NO REPORTADO,2010-01-01 00:00:00,MASCULINO,ADULTOS,1,abigeato,NaN,2010-01-01 00:00:00
2,BOYACÁ,BELÉN,15087000,NO REPORTADO,2010-01-02 00:00:00,FEMENINO,ADULTOS,1,abigeato,NaN,2010-01-02 00:00:00
3,CASANARE,PORE,85263000,NO REPORTADO,2010-01-02 00:00:00,MASCULINO,ADULTOS,1,abigeato,NaN,2010-01-02 00:00:00
4,CALDAS,NEIRA,17486000,NO REPORTADO,2010-01-03 00:00:00,MASCULINO,ADULTOS,1,abigeato,NaN,2010-01-03 00:00:00


In [196]:
full_df[full_df['CRIME'] == 'secuestro']

,DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,ARMAS_MEDIOS,FECHA_HECHO,GENERO,GRUPO_ETARIO,CANTIDAD,CRIME,DESCRIPCION_CONDUCTA
0,ANTIOQUIA,APARTADÓ,05045000,ARMA BLANCA / CORTOPUNZANTE,2010-07-23 00:00:00,MASCULINO,ADULTOS,1,secuestro,ARTÍCULO 169. SECUESTRO EXTORSIVO
1,ANTIOQUIA,BELLO,05088000,ARMA BLANCA / CORTOPUNZANTE,2010-03-04 00:00:00,MASCULINO,ADULTOS,1,secuestro,ARTÍCULO 169. SECUESTRO EXTORSIVO
2,ANTIOQUIA,BRICEÑO,05107000,ARMA BLANCA / CORTOPUNZANTE,2010-01-05 00:00:00,MASCULINO,ADULTOS,1,secuestro,ARTÍCULO 169. SECUESTRO EXTORSIVO
3,ANTIOQUIA,CÁCERES,05120000,ARMA BLANCA / CORTOPUNZANTE,2010-08-23 00:00:00,MASCULINO,ADULTOS,1,secuestro,ARTÍCULO 169. SECUESTRO EXTORSIVO
4,ANTIOQUIA,CAÑASGORDAS,05138000,ARMA BLANCA / CORTOPUNZANTE,2010-05-05 00:00:00,MASCULINO,ADULTOS,2,secuestro,ARTÍCULO 169. SECUESTRO EXTORSIVO
...,...,...,...,...,...,...,...,...,...,...
82,VALLE,CALI (CT),76001000,ARMA DE FUEGO,2020-11-28 00:00:00,MASCULINO,ADULTOS,1,secuestro,NaN
83,ANTIOQUIA,MEDELLÍN (CT),05001000,SIN EMPLEO DE ARMAS,2020-12-03 00:00:00,MASCULINO,ADULTOS,1,secuestro,NaN
84,VALLE,CALI (CT),76001000,SIN EMPLEO DE ARMAS,2020-12-04 00:00:00,MASCULINO,ADULTOS,1,secuestro,NaN
85,CAUCA,TORIBIO,19821000,ARMA DE FUEGO,2020-12-05 00:00:00,MASCULINO,ADULTOS,1,secuestro,NaN


# summaries

In [189]:
all_crimes = full_df.groupby(['DEPARTAMENTO','MUNICIPIO','CODIGO_DANE'])['CANTIDAD'].sum().reset_index().sort_values('CANTIDAD', ascending=False)
all_crimes

,DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,CANTIDAD
493,CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,1346081
1064,VALLE,CALI (CT),76001000,414010
70,ANTIOQUIA,MEDELLÍN (CT),05001000,376321
135,ATLÁNTICO,BARRANQUILLA (CT),08001000,183681
904,SANTANDER,BUCARAMANGA (CT),68001000,132584
...,...,...,...,...
372,CASANARE,LA SALINA,85136000,39
939,SANTANDER,JORDÁN,68370000,30
814,NO REGISTRA,NO REGISTRA,NO REGISTRA,3
748,NARIÑO,-,-,3


In [190]:
all_crimes.to_csv('summaries/all_crimes.csv',index=False,encoding='utf8',sep=';')

In [191]:
by_crime = full_df.groupby(['DEPARTAMENTO','MUNICIPIO','CODIGO_DANE','CRIME'])['CANTIDAD'].sum().reset_index().sort_values('CANTIDAD', ascending=False)
by_crime

,DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,CRIME,CANTIDAD
7186,CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,hurto_a_personas,567788
7195,CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,violencia_intrafamiliar,199840
7191,CUNDINAMARCA,BOGOTÁ D.C. (CT),11001000,lesiones_personales,183680
15550,VALLE,CALI (CT),76001000,hurto_a_personas,122658
1073,ANTIOQUIA,MEDELLÍN (CT),05001000,hurto_a_personas,119947
...,...,...,...,...,...
8773,CÓRDOBA,BUENAVISTA,23079000,pirateria_terrestre,1
14546,SUCRE,SAMPUÉS,70670000,secuestro,1
13749,SANTANDER,LA PAZ,68397000,hurto_motocicletas,1
10339,MAGDALENA,PLATO,47555000,secuestro,1


In [192]:
by_crime.to_csv('summaries/by_crime.csv',index=False,encoding='utf8',sep=';')

In [156]:
full_df['FECHA'] = pd.to_datetime(full_df['FECHA_HECHO'], format='%Y-%m-%d ', errors='ignore')

In [157]:
full_df.dtypes

DEPARTAMENTO            object
MUNICIPIO               object
CODIGO_DANE             object
ARMAS_MEDIOS            object
FECHA_HECHO             object
GENERO                  object
GRUPO_ETARIO            object
CANTIDAD                 int64
DESCRIPCION_CONDUCTA    object
FECHA                   object
dtype: object

In [158]:
full_df.head()

,DEPARTAMENTO,MUNICIPIO,CODIGO_DANE,ARMAS_MEDIOS,FECHA_HECHO,GENERO,GRUPO_ETARIO,CANTIDAD,DESCRIPCION_CONDUCTA,FECHA
0,HUILA,CAMPOALEGRE,41132000,NO REPORTADO,2010-01-01 00:00:00,FEMENINO,ADULTOS,1,NaN,2010-01-01 00:00:00
1,META,SAN JUAN DE ARAMA,50683000,NO REPORTADO,2010-01-01 00:00:00,MASCULINO,ADULTOS,1,NaN,2010-01-01 00:00:00
2,BOYACÁ,BELÉN,15087000,NO REPORTADO,2010-01-02 00:00:00,FEMENINO,ADULTOS,1,NaN,2010-01-02 00:00:00
3,CASANARE,PORE,85263000,NO REPORTADO,2010-01-02 00:00:00,MASCULINO,ADULTOS,1,NaN,2010-01-02 00:00:00
4,CALDAS,NEIRA,17486000,NO REPORTADO,2010-01-03 00:00:00,MASCULINO,ADULTOS,1,NaN,2010-01-03 00:00:00


In [152]:
full_df.to_parquet('police_data/all_data.parquet')

ValueError: Can't infer object conversion type: 0    2010-01-01 00:00:00
1    2010-01-01 00:00:00
2    2010-01-02 00:00:00
3    2010-01-02 00:00:00
4    2010-01-03 00:00:00
5    2010-01-03 00:00:00
6    2010-01-03 00:00:00
7    2010-01-04 00:00:00
8    2010-01-04 00:00:00
9    2010-01-04 00:00:00
Name: FECHA_HECHO, dtype: object